In [3]:
import streamlit as st
from streamlit_option_menu import option_menu
import plotly.express as px
import pandas as pd
import psycopg2
import requests
import json

2025-01-18 21:57:32.043 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [2]:
#SQL CONNECTION

mydb=psycopg2.connect(host='localhost',
                        user='postgres',
                        password='1234',
                        database='phonepe_data',
                        port='5432')
cursor=mydb.cursor()

#agg_insurance_df

cursor.execute('SELECT * FROM aggregate_insurance')
mydb.commit()
tabel1=cursor.fetchall()

agg_insurance=pd.DataFrame(tabel1,columns=('States', 'Years', 'Quarter', 'Transaction_type', 'Transaction_count',
       'Transaction_amount'))

#agg_transaction_df

cursor.execute('SELECT * FROM aggregate_transaction')
mydb.commit()
tabel2=cursor.fetchall()

agg_transaction=pd.DataFrame(tabel2,columns=('States', 'Years', 'Quarter', 'Transaction_type', 'Transaction_count',
       'Transaction_amount'))

#agg_user_df

cursor.execute('SELECT * FROM aggregate_user')
mydb.commit()
tabel3=cursor.fetchall()

agg_user=pd.DataFrame(tabel3,columns=('States', 'Years', 'Quarter', 'Brands', 'Transaction_count',
       'Percentage'))

#map_insurance_df

cursor.execute('SELECT * FROM map_insurance')
mydb.commit()
tabel4=cursor.fetchall()

map_insurance=pd.DataFrame(tabel4,columns=('States', 'Years', 'Quarter', 'Districts', 'Transaction_count',
       'Transaction_amount'))

#map_transaction_df

cursor.execute('SELECT * FROM map_transaction')
mydb.commit()
tabel5=cursor.fetchall()

map_transaction=pd.DataFrame(tabel5,columns=('States', 'Years', 'Quarter', 'Districts', 'Transaction_count',
       'Transaction_amount'))

#map_user_df

cursor.execute('SELECT * FROM map_user')
mydb.commit()
tabel6=cursor.fetchall()

map_user=pd.DataFrame(tabel6,columns=('States', 'Years', 'Quarter', 'Districts', 'RegisteredUsers',
       'AppOpens'))

#top_insurance_df

cursor.execute('SELECT * FROM top_insurance')
mydb.commit()
tabel7=cursor.fetchall()

top_insurance=pd.DataFrame(tabel7,columns=('States', 'Years', 'Quarter', 'Pincodes', 'Transaction_count',
       'Transaction_amount'))

#top_transaction_df

cursor.execute('SELECT * FROM top_transaction')
mydb.commit()
tabel8=cursor.fetchall()

top_transaction=pd.DataFrame(tabel8,columns=('States', 'Years', 'Quarter', 'Pincodes', 'Transaction_count',
       'Transaction_amount'))

#top_user_df

cursor.execute('SELECT * FROM top_user')
mydb.commit()
tabel9=cursor.fetchall()

top_user=pd.DataFrame(tabel9,columns=('States', 'Years', 'Quarter', 'Pincodes', 'RegisteredUsers'))

In [3]:
top_user

,States,Years,Quarter,Pincodes,RegisteredUsers
0,Andaman & Nicobar,2018,1,744103,1608
1,Andaman & Nicobar,2018,1,744101,1108
2,Andaman & Nicobar,2018,1,744105,1075
3,Andaman & Nicobar,2018,1,744102,1006
4,Andaman & Nicobar,2018,1,744104,272
...,...,...,...,...,...
9277,West Bengal,2024,2,700015,136835
9278,West Bengal,2024,2,721101,132872
9279,West Bengal,2024,2,742304,131799
9280,West Bengal,2024,2,700150,128721


In [4]:
#Transaction year Based

def transaction_amount_count_Y(df,year):
    tacy=df[df['Years'] == year]
    tacy.reset_index(drop=True,inplace=True)

    tacyg=tacy.groupby('States')[['Transaction_count','Transaction_amount']].sum()
    tacyg.reset_index(inplace=True)

    col1,col2=st.columns(2)

    with col1:

        fig_amount=px.bar(tacyg,x='States',y='Transaction_amount',title=f'{year} TRANSACTION AMOUNT',
                            color_discrete_sequence=px.colors.sequential.Aggrnyl, height=650, width=600)
        st.plotly_chart(fig_amount)
    
    with col2:
    
        fig_count=px.bar(tacyg,x='States',y='Transaction_count',title=f'{year} TRANSACTION COUNT',
                        color_discrete_sequence=px.colors.sequential.Turbo, height=650, width=600)
        st.plotly_chart(fig_count)

    url='https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson'
    response=requests.get(url)
    data1=json.loads(response)
    states_name=[]
    for feature in data1['features']:
        states_name.append(feature['properties']['ST_NM'])
    states_name.sort()

    fig_india_1=px.choropleth(tacyg,geojson=data1,locations='States',featureidkey='properties.ST_NM',color='Transaction_amount',
                              color_continuous_scale='Rainbow',range_color=(tacyg['Transaction_amount'].min(),tacyg['Transaction_amount'].min()),
                              hover_name='States',title=f'{year} TRANSACTION AMOUNT',fitbounds='locations',height=600,width=600)
    fig_india_1.update.geos(visible=False)
    fig_india_1.show()

    fig_india_2=px.choropleth(tacyg,geojson=data1,locations='States',featureidkey='properties.ST_NM',color='Transaction_count',
                              color_continuous_scale='Rainbow',range_color=(tacyg['Transaction_count'].min(),tacyg['Transaction_count'].min()),
                              hover_name='States',title=f'{year} TRANSACTION COUNT',fitbounds='locations',height=600,width=600)
    fig_india_2.update.geos(visible=False)
    fig_india_2.show()

In [5]:
transaction_amount_count_Y(2024)

TypeError: transaction_amount_count_Y() missing 1 required positional argument: 'year'

In [8]:
transaction_amount_count_Y

<function __main__.transaction_amount_count_Y(df, year)>

In [9]:
#Transaction Quarter Based


def transaction_amount_count_Y_Q(df,quarter):
    tacy=df[df['Quarter'] == quarter]
    tacy.reset_index(drop=True,inplace=True)

    tacyg=tacy.groupby('States')[['Transaction_count','Transaction_amount']].sum()
    tacyg.reset_index(inplace=True)

    col1,col2=st.columns(2)

    with col1:

        fig_amount=px.bar(tacyg,x='States',y='Transaction_amount',title=f'{tacy["Years"].min()} YEAR {quarter} QUARTER TRANSACTION AMOUNT',
                            color_discrete_sequence=px.colors.sequential.Aggrnyl, height=650, width=600)
        st.plotly_chart(fig_amount)
    
    with col2:
    
        fig_count=px.bar(tacyg,x='States',y='Transaction_count',title=f'{tacy["Years"].min()} YEAR {quarter} QUARTER TRANSACTION AMOUNT',
                        color_discrete_sequence=px.colors.sequential.Turbo, height=650, width=600)
        st.plotly_chart(fig_count)

    url='https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson'
    response=requests.get(url)
    data1=json.loads(response.content)
    states_name=[]
    for feature in data1['features']:
        states_name.append(feature['properties']['ST_NM'])
    states_name.sort()

    col1,col2=st.columns(2)

    with col1:

        fig_india_1=px.choropleth(tacyg,geojson=data1,locations='States',featureidkey='properties.ST_NM',color='Transaction_amount',
                                color_continuous_scale='Rainbow',range_color=(tacyg['Transaction_amount'].min(),tacyg['Transaction_amount'].min()),
                                hover_name='States',title=f'{tacy["Years"].min()} YEAR {quarter} QUARTER TRANSACTION AMOUNT',fitbounds='locations',height=600,width=600)
        fig_india_1.update_geos(visible=False)
        st.plotly_chart(fig_india_1)
        
    with col2:   

        fig_india_2=px.choropleth(tacyg,geojson=data1,locations='States',featureidkey='properties.ST_NM',color='Transaction_count',
                                color_continuous_scale='Rainbow',range_color=(tacyg['Transaction_count'].min(),tacyg['Transaction_count'].min()),
                                hover_name='States',title=f'{tacy["Years"].min()} YEAR {quarter} QUARTER TRANSACTION AMOUNT',fitbounds='locations',height=600,width=600)
        fig_india_2.update_geos(visible=False)
        st.plotly_chart(fig_india_2)

In [10]:
#Transaction State based Based

def Agg_Transaction_Type(df,state):
    tacy=df[df['States'] == state]
    tacy.reset_index(drop=True,inplace=True)

    tacyg=tacy.groupby('Transaction_type')[['Transaction_count','Transaction_amount']].sum()
    tacyg.reset_index(inplace=True)

    col1,col2=st.columns(2)

    with col1:

        fig_amount=px.pie(data_frame=tacyg, names='Transaction_type', values='Transaction_amount',
                          width=600, title=f'{state.upper()} TRANSACTION AMOUNT', hole=0.5)
        st.plotly_chart(fig_amount)
    
    with col2:
    
        fig_count=px.pie(data_frame=tacyg, names='Transaction_type', values='Transaction_count',
                          width=600, title=f'{state.upper()} TRANSACTION COUNT', hole=0.5)
        st.plotly_chart(fig_count)

In [11]:
#Agg_User_Analyziz_1

def Agg_User_Plot_1(df,year):
    aguy=df[df['Years'] == year]
    aguy.reset_index(drop=True,inplace=True)

    aguyg=pd.DataFrame(aguy.groupby('Brands')['Transaction_count'].sum())
    aguyg.reset_index(inplace=True)

    fig_bar_1=px.bar(aguyg, x='Brands', y= 'Transaction_count', title=f'{year} BRANDS AND TRANSACTION COUNT',
                     width= 1000, color_discrete_sequence=px.colors.sequential.haline_r, hover_name='Brands')
    
    st.plotly_chart(fig_bar_1)

    return aguy

In [12]:
#Agg_User_Analyziz_2

def Agg_User_Plot_2(df,quarter):
    aguyg=df[df['Quarter'] == quarter]
    aguyg.reset_index(drop=True,inplace=True)

    aguygq=pd.DataFrame(aguyg.groupby('Brands')['Transaction_count'].sum())
    aguygq.reset_index(inplace=True)

    fig_bar_1=px.bar(aguygq, x='Brands', y= 'Transaction_count', title=f'{quarter} QUARTER, BRANDS AND TRANSACTION COUNT',
                     width= 1000, color_discrete_sequence=px.colors.sequential.haline_r, hover_name='Brands')
    
    st.plotly_chart(fig_bar_1)

    return aguyg


In [6]:
#Agg_User_Analyziz_2

def Agg_User_Plot_3(df,state):
    auyqs=df[df['States'] == state]
    auyqs.reset_index(drop=True,inplace=True)

    fig_line_1=px.line(auyqs, x='Brands', y= 'Transaction_count', title=f'{state.upper()} BRANDS, TRANSACTION COUNT AND PERCENTAGE',
                     width= 1000, hover_name='Percentage', markers=True)
    
    st.plotly_chart(fig_line_1)

In [7]:
#Map User Plot 1

def map_User_Plot_1(df,year):
    muy=df[df['Years'] == year]
    muy.reset_index(drop=True,inplace=True)

    muyg=muy.groupby('States')[['RegisteredUsers','AppOpens']].sum()
    muyg.reset_index(inplace=True)

    fig_bar_1=px.line(muyg, x='States', y= ['RegisteredUsers','AppOpens'], title=f'{year} REGISTERED USER AND APP OPENS',
                     width= 1000,height=600, markers=True)
    
    st.plotly_chart(fig_bar_1)

    return muy

In [8]:
#Map User Plot 2

def map_User_Plot_2(df,quarter):
    muyq=df[df['Quarter'] == quarter]
    muyq.reset_index(drop=True,inplace=True)

    muyqg=muyq.groupby('States')[['RegisteredUsers','AppOpens']].sum()
    muyqg.reset_index(inplace=True)

    fig_bar_1=px.line(muyqg, x='States', y= ['RegisteredUsers','AppOpens'], title=f'{df['Years'].min()} YEAR {quarter} QUARTER REGISTERED USER AND APP OPENS',
                     width= 1000,height=600, markers=True, color_discrete_sequence=px.colors.sequential.Rainbow_r)
    
    st.plotly_chart(fig_bar_1)

    return muyq

In [9]:
#Map User Plot 3

def map_user_plot_3(df,state):
    muyqs=df[df['States'] == state]
    muyqs.reset_index(drop=True,inplace=True)

    col1,col2=st.columns(2)

    with col1:

        fig_map_user_bar_1=px.bar(muyqs, x='RegisteredUsers', y= 'Districts', orientation='h',
                                  title=f'{state.upper()} REGISTERED USER', height=800, color_discrete_sequence=px.colors.sequential.Brwnyl)
        st.plotly_chart(fig_map_user_bar_1)
    
    with col2:
    
        fig_map_user_bar_2=px.bar(muyqs, x='AppOpens', y= 'Districts', orientation='h',
                                  title=f'{state.upper()} APPOPENS', height=800, color_discrete_sequence=px.colors.sequential.Darkmint)
        st.plotly_chart(fig_map_user_bar_2)

In [13]:
def top_insur_plot_1(df,state):
    tiy=df[df['States'] == state]
    tiy.reset_index(drop=True,inplace=True)

    col1,col2=st.columns(2)

    with col1:

        fig_top_insur_bar_1=px.bar(tiy, x='Quarter', y= 'Transaction_amount', hover_data='Pincodes',
                                  title=f'{state.upper()} TRANSACTION AMOUNT', height=650,width=600, color_discrete_sequence=px.colors.sequential.amp)
        st.plotly_chart(fig_top_insur_bar_1)
    
    with col2:

        fig_top_insur_bar_2=px.bar(tiy, x='Quarter', y= 'Transaction_count', hover_data='Pincodes',
                                  title=f'{state.upper()} TRANSACTION COUNT', height=650,width=600, color_discrete_sequence=px.colors.sequential.BuGn)
        st.plotly_chart(fig_top_insur_bar_2)

In [1]:
#top_User_plot_1

def top_User_Plot_1(df,year):
    tuy=df[df['Years'] == year]
    tuy.reset_index(drop=True,inplace=True)

    tuyg=pd.DataFrame(tuy.groupby(['States','Quarter'])['RegisteredUsers'].sum())
    tuyg.reset_index(inplace=True)

    fig_top_plot_1=px.bar(tuyg, x='States', y= 'RegisteredUsers', title=f'{year} REGISTERED USER',
                     width= 1000, height=800, color_discrete_sequence=px.colors.sequential.Blugrn_r,hover_name='States',color='Quarter')
    
    st.plotly_chart(fig_top_plot_1)

    return tuy

In [2]:
#top_User__plot_2

def top_User_Plot_2(df,state):
    tuys=df[df['States'] == state]
    tuys.reset_index(drop=True,inplace=True)

    fig_top_plot_=px.bar(tuys, x='Quarter', y= 'RegisteredUsers', title=f'{state} REGISTERED USER, PINCODES AND QUARTER',
                     width= 1000, height=800, color_discrete_sequence=px.colors.sequential.Magenta_r,hover_data='Pincodes',color='RegisteredUsers')
    
    st.plotly_chart(fig_top_plot_)

In [4]:
#SQL CONNECTION
def top_chart_transaction_amount(table_name):
    mydb=psycopg2.connect(host='localhost',
                            user='postgres',
                            password='1234',
                            database='phonepe_data',
                            port='5432')
    cursor=mydb.cursor()

    #plot_1

    query1=f''' SELECT states, SUM(transaction_amount) AS  transaction_amount FROM {table_name} GROUP BY states
                ORDER BY transaction_amount DESC LIMIT 10;'''

    cursor.execute(query1)
    tabel1=cursor.fetchall()
    mydb.commit()

    df1=pd.DataFrame(tabel1, columns=('states','transaction_amount'))

    fig_amount_1=px.bar(df1,x='states',y='transaction_amount', title='TRANSACTION AMOUNT',hover_name='states', color_discrete_sequence= px.colors.sequential.Aggrnyl,
                        height=650, width=600)

    st.plotly_chart(fig_amount_1)

    #plot_2

    query2=f''' SELECT states, SUM(transaction_amount) AS  transaction_amount FROM {table_name} GROUP BY states
                ORDER BY transaction_amount LIMIT 10;'''

    cursor.execute(query2)
    tabel2=cursor.fetchall()
    mydb.commit()

    df2=pd.DataFrame(tabel2, columns=('states','transaction_amount'))

    fig_amount_2=px.bar(df2,x='states',y='transaction_amount', title='TRANSACTION AMOUNT',hover_name='states', color_discrete_sequence= px.colors.sequential.Aggrnyl_r,
                        height=650, width=600)

    st.plotly_chart(fig_amount_2)

    #plot_3 

    query3=f''' SELECT states, SUM(transaction_amount) AS  transaction_amount FROM {table_name} GROUP BY states
                ORDER BY transaction_amount;'''

    cursor.execute(query3)
    tabel3=cursor.fetchall()
    mydb.commit()

    df3=pd.DataFrame(tabel3, columns=('states','transaction_amount'))

    fig_amount_3=px.bar(df3,x='transaction_amount',y='states', title='TRANSACTION AMOUNT',hover_name='states', color_discrete_sequence= px.colors.sequential.Bluered_r,
                        height=1000, width=800)

    st.plotly_chart(fig_amount_3)

###Aggregated_User

In [10]:
Agg_User_Y=Agg_User_Plot_1(agg_user,2021)

NameError: name 'Agg_User_Plot_1' is not defined

In [11]:
Agg_user_Y_Q = Agg_User_Plot_2(Agg_User_Y,2)

NameError: name 'Agg_User_Plot_2' is not defined

In [12]:
Agg_user_Y_Q

NameError: name 'Agg_user_Y_Q' is not defined

In [17]:
map_user

,States,Years,Quarter,Districts,RegisteredUsers,AppOpens
0,Andaman & Nicobar,2018,1,north and middle andaman district,632,0
1,Andaman & Nicobar,2018,1,south andaman district,5846,0
2,Andaman & Nicobar,2018,1,nicobars district,262,0
3,Andaman & Nicobar,2018,2,north and middle andaman district,911,0
4,Andaman & Nicobar,2018,2,south andaman district,8143,0
...,...,...,...,...,...,...
19027,West Bengal,2024,2,nadia district,1737326,14549939
19028,West Bengal,2024,2,birbhum district,1099269,38692858
19029,West Bengal,2024,2,purba medinipur district,1721598,18370486
19030,West Bengal,2024,2,maldah district,1231444,42755282
